## An example of basic MLIR
### Some notes: 
On my machine I have LLVM 20.1.8 and supports mlir
( /opt/homebrew/opt/llvm/bin/mlir-opt --version
Homebrew LLVM version 20.1.8
  Optimized build.) 

ls /opt/homebrew/opt/llvm/bin | grep mlir

-->mlir-cat
-->mlir-linalg-ods-yaml-gen #generate Linalg dialect ops from ODS/YAML definitions (more advanced).
-->mlir-lsp-server #language server for MLIR editing
-->mlir-minimal-opt #small mlir-opt build for minimal pipelines
-->mlir-minimal-opt-canonicalize
-->mlir-opt #run passes
-->mlir-pdll #pattern definition language tool
-->mlir-pdll-lsp-server
-->mlir-query #query and analyze MLIR IR.
-->mlir-reduce #reduce/trim failing IR testcases
-->mlir-rewrite
-->mlir-runner #execute MLIR via JIT
-->mlir-tblgen #generate dialect/op definitions from TableGen
-->mlir-transform-opt #transformation-specific toolchain
-->mlir-translate #convert between MLIR and other formats (e.g., LLVM IR).

##MLIR sprang from the need to answer: Could we build a unified representation that could support every AI framework, every hardware backend, and every kind of optimization—from algebraic simplification to polyhedral analysis? 
MLIR dialects — a way to cleanly separate domain-specific concerns from the core infrastructure of a compiler. Rather than forcing every user to adopt a rigid, one-size-fits-all intermediate representation (like LLVM and other compilers), MLIR would let compiler engineers define their own representations—custom ops, types, and semantics—tailored to their domain. AI-specific dialects—including arith, linalg, and tensor—providing some bits and pieces useful for building a modern AI compiler stack.

#https://mlir.llvm.org/docs/Tutorials/Toy/
### Some links:
#Chris Lattner & Tatiana Shpeisman talk in 2019 LLVM Developer meeting: https://www.youtube.com/watch?v=qzljG6DKgic

#https://www.stephendiehl.com/posts/mlir_introduction/

#https://www.youtube.com/watch?v=cyICUIZ56wQ

#https://www.modular.com/blog/democratizing-ai-compute-part-8-what-about-the-mlir-compiler-infrastructure

#https://www.modular.com/blog/democratizing-ai-compute-part-6-what-about-ai-compilers

#https://veitner.bearblog.dev/let-the-compiler-do-the-work-in-cutedsl/

In [25]:
# Write MLIR code - a simple toy example to observe llvm IR generation
#****************************************************
#func.func — entry point in the func dialect (registered by default).
#arith.addi — integer addition in arith dialect (also available in your build).
#canonicalize — cleans up IR before lowering.
#convert-arith-to-llvm — lowers arith ops into llvm dialect equivalents.
#convert-func-to-llvm — lowers func dialect to llvm dialect functions.
#mlir-translate --mlir-to-llvmir — converts the MLIR llvm dialect to textual LLVM IR.

#my pipeline here is:
#MLIR (func + arith)
#   ↓ canonicalize
#   ↓ convert-arith-to-llvm
#   ↓ convert-func-to-llvm
#MLIR (llvm dialect)
#   ↓ mlir-translate
#LLVM IR (.ll)

#mlir doesnot have a code generator for target assembly. LLVM does, so we convert all the different
#dialects to LLVM IR
#****************************************************

mlir_code = """\
module  {
  func.func @add(%a: i32, %b: i32) -> i32 {
    %sum = arith.addi %a, %b : i32
    return %sum : i32
  }
}
"""

with open("add.mlir", "w") as f:
    f.write(mlir_code)

# Combine the optimization and lowering steps into a single pass pipeline.
# This pipeline now includes all the necessary passes to convert the `func` dialect to LLVM.
!mlir-opt add.mlir --pass-pipeline="builtin.module(func.func(canonicalize, convert-arith-to-llvm), convert-func-to-llvm)" | mlir-translate --mlir-to-llvmir - > add.ll

# Print the generated LLVM IR
!head -40 add.ll


; ModuleID = 'LLVMDialectModule'
source_filename = "LLVMDialectModule"

define i32 @add(i32 %0, i32 %1) {
  %3 = add i32 %0, %1
  ret i32 %3
}

!llvm.module.flags = !{!0}

!0 = !{i32 2, !"Debug Info Version", i32 3}


In [26]:
#Run the LLVM IR through llc to get assembly.
#LLVM IR (.ll)
#   ↓ llc
#Assembly (.s)

!llc add.ll -o add.s
!head -20 add.s


	.build_version macos, 15, 0
	.section	__TEXT,__text,regular,pure_instructions
	.globl	_add                            ; -- Begin function add
	.p2align	2
_add:                                   ; @add
	.cfi_startproc
; %bb.0:
	add	w0, w0, w1
	ret
	.cfi_endproc
                                        ; -- End function
.subsections_via_symbols


In [27]:
#run assembly through clang to get an executable
#Assembly (.s)
#   ↓ clang
#Executable


driver_code = r"""
#include <stdio.h>

// Declare the MLIR-generated function
int add(int a, int b);

int main() {
    int result = add(42, 58);
    printf("Result: %d\n", result);
    return 0;
}
"""

with open("driver.c", "w") as f:
    f.write(driver_code)

# Compile: assembly + driver → executable
!clang add.s driver.c -o add_exec


In [28]:
#Run the executable
!./add_exec

Result: 100


In [ ]:
Registered dialects on my mlir version: acc, amx, arm_neon, arm_sme, arm_sve, builtin, dlti, func, gpu, llvm, nvvm, omp, rocdl, spirv, vcix, x86vector ; for more info on dialect registration see https://mlir.llvm.org/getting_started/Faq/#registered-loaded-dependent-whats-up-with-dialects-management